# 데이터 to CSV

In [ ]:
import cv2
import mediapipe as mp
import math
import numpy as np
import pandas as pd
import os
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

## 2. Path 설정
1. 숫자
2. 영어

In [ ]:
path_dir = './Sign-Language-Digits-Dataset-master/Sign Language for Numbers/'
#path_dir = './dataset/'
# 0 ... 9 손가락 숫자
path_type = [i for i in range(10)]
decimal_path_type = [i for i in range(10)]
alphabet_path_type = [i for i in range(97,123)]

## 3. 데이터 처리 형식 지정
- 각도 변환

In [ ]:
def joint_to_angle(landmark):
    
    # 21 x,y,z np array 생성
    joint = np.zeros((21,3))
    
    for j,lm in enumerate(landmark.landmark):
        joint[j] = [lm.x,lm.y,lm.z]
    
    joint1=  joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19],:]
    joint2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],:]

    # 각 좌표의 벡터를 구함
    vec = joint1 - joint2

    vec = vec / np.linalg.norm(vec,axis=1)[:,np.newaxis]
    # 기존 벡터 변환
#     compareV1 = vec[[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17],:]
#     compareV2 = vec[[1, 2, 3, 5, 6, 7 ,9, 10, 11, 13, 14, 15, 17, 18, 19],:]

    # 새로운 벡터 변환
    compareV1 = vec[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17, 17, 18],:]
    compareV2 = vec[[1, 2, 3, 7, 5, 6, 7, 9, 9, 10, 11, 13, 14, 15, 17, 18, 19, 19],:]
    angle = np.arccos(np.einsum('nt,nt->n',compareV1,compareV2))

    angle = np.degrees(angle)
    
    return angle

## 4. 데이터 추출

In [ ]:
def image_to_data(path_type):
    # For static images:
    # TODO 여기 안에 처리되는 데이터를, 배열로 저장해서 CSV 형태로 저장

    landmarks_list = list(list())

    # 0 ~ 20
    # x, y, z
    # 63
    # 63 + target[0 ~ 9]

    # 데이터가 0~9 까지 각각 200개씩
    # 최댓값은 2000개

    for t in path_type:
        IMAGE_FILES = list()
        # 불러오는 위치
        from_path = path_dir + str(t) + '/'

        # 파일 불러오기
        file_list = os.listdir(from_path)
        # 확장자가 jpg 인 모든 파일을 불러온다.
        IMAGE_FILES+=[from_path+ file for file in file_list if file.endswith('.JPG') or file.endswith('.jpg')]

        with mp_hands.Hands(static_image_mode = True,max_num_hands = 1,min_detection_confidence=0.5) as hands:
            for idx, file in enumerate(IMAGE_FILES):
                #print(file)

                idx_landmarks = []

                image = cv2.flip(cv2.imread(file), 0)

                result = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                if not result.multi_hand_landmarks:
                    continue

                image_hegiht, image_width, _ =image.shape

                annotated_image = image.copy()
                for hand_landmarks in result.multi_hand_landmarks:
                    print('now : ',t)
                    #landmarks_list.append(np.append(joint_to_angle(hand_landmarks),int(t)))
                    landmarks_list.append(np.append(joint_to_angle(hand_landmarks),t))
                
    return landmarks_list

In [ ]:
landmarks_list = image_to_data(decimal_path_type)

## 5. 차원 수 만큼 할당

In [ ]:
# d : 차원 수
def df_to_csv(d,file_name):
    df_columns = []
    for i in range(d):
        df_columns.append(i)


    print(df_columns)

    df_columns.append('target')

    # print(landmarks_list)
    df = pd.DataFrame(landmarks_list,columns=df_columns)
    df.columns
    print(df)
    df.to_csv(file_name)
    return df

In [ ]:
df_to_csv(18,'output_digit.csv')

## 6. CSV 로 내보내기

# CSV to TensorFlow

In [ ]:
import functools

import numpy as np

import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split

In [ ]:
TRAIN_DATA_URL = './output_alpha.csv'

data = pd.read_csv(TRAIN_DATA_URL)

In [ ]:
data_train, data_test = train_test_split(data, 
                 test_size=1/10,
                stratify=data['target'],
                random_state=20210918)

target_train = data_train.pop('target')
target_test = data_test.pop('target')

data_train.pop('Unnamed: 0')
data_test.pop('Unnamed: 0')

## 모델 생성
- 시작 값(차원 수)
- 끝 값(분류할 차원 수)

In [ ]:
def get_compiled_model4():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64,activation='relu',input_shape=(15,)))
    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.Dense(32,activation='relu'))
    model.add(tf.keras.layers.Dense(26,activation='softmax'))
    
    model.summary()
    
    model.compile(optimizer='Adam',
                 loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])
    
    return model


## 학습

In [ ]:
# model = get_compiled_model()
# model.fit(dataset_train,epochs=10)
model = get_compiled_model4()
# model.summary()

# model.compile(optimizer=tf.optimizers.Adam(),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])


#(x_train, y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()
print(target_train)
t = []
for a in target_train:
    tmplist = list()
    for i in range(26):
        tmplist.append(0)
    tmplist[ord(a)-97] = 1
    t.append(tmplist)
print(t)

model.fit(np.array(data_train),
          np.array(t), epochs=20)

In [ ]:
model.save('OutputModel_Alpha')

# 실행

In [ ]:
import tensorflow
import mediapipe as mp
import cv2
import time
import numpy as np
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils

# 

sequence = []
sentence = []
threshold = 0.4
actions=['0','1','2','3','4','5','6','7','8','9']
actions_alpha=[chr(i) for i in range(65,91)]

#model = tensorflow.keras.models.load_model('./OutputModel_Alpha')

In [ ]:
def joint_to_angle(landmark):
    
    # 21 x,y,z np array 생성
    joint = np.zeros((21,3))
    
    for j,lm in enumerate(landmark.landmark):
        joint[j] = [lm.x,lm.y,lm.z]
    
    joint1=  joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19],:]
    joint2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],:]

    # 각 좌표의 벡터를 구함
    vec = joint1 - joint2

    vec = vec / np.linalg.norm(vec,axis=1)[:,np.newaxis]
    
    compareV1 = vec[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17, 17,18],:]
    compareV2 = vec[[1, 2, 3, 7, 5, 6, 7, 9, 9, 10, 11, 13, 14, 15, 17, 18, 19,19],:]
#     compareV1 = vec[[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17],:]
#     compareV2 = vec[[1, 2, 3, 5, 6, 7 ,9, 10, 11, 13, 14, 15, 17, 18, 19],:]
    angle = np.arccos(np.einsum('nt,nt->n',compareV1,compareV2))

    angle = np.degrees(angle)
    
    return angle

def joint_to_angle_alpha(landmark):
    
    # 21 x,y,z np array 생성
    joint = np.zeros((21,3))
    
    for j,lm in enumerate(landmark.landmark):
        joint[j] = [lm.x,lm.y,lm.z]
    
    joint1=  joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19],:]
    joint2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],:]

    # 각 좌표의 벡터를 구함
    vec = joint1 - joint2

    vec = vec / np.linalg.norm(vec,axis=1)[:,np.newaxis]
    
#     compareV1 = vec[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17, 17,18],:]
#     compareV2 = vec[[1, 2, 3, 7, 5, 6, 7, 9, 9, 10, 11, 13, 14, 15, 17, 18, 19,19],:]
    compareV1 = vec[[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17],:]
    compareV2 = vec[[1, 2, 3, 5, 6, 7 ,9, 10, 11, 13, 14, 15, 17, 18, 19],:]
    angle = np.arccos(np.einsum('nt,nt->n',compareV1,compareV2))

    angle = np.degrees(angle)
    
    return angle



In [ ]:
TRAIN_DATA_URL = './output_digit.csv'

file = np.genfromtxt(TRAIN_DATA_URL,delimiter=',')
file = np.delete(file,0,axis=1)
file = np.delete(file,0,axis=0)
angleFile = file[1:,:-1]
labelFile = file[1:,-1]

angle = angleFile.astype(np.float32)
label = labelFile.astype(np.float32)
knn = cv2.ml.KNearest_create()
knn.train(angle,cv2.ml.ROW_SAMPLE,label)

file_alpha = np.genfromtxt('./DataSet.txt',delimiter=',')
angleFile_alpha = file_alpha[:,:-1]
labelFile_alpha = file_alpha[:,-1]                         

angle_alpha = angleFile_alpha.astype(np.float32)
label_alpha = labelFile_alpha.astype(np.float32)
knn_alpha = cv2.ml.KNearest_create()
knn_alpha.train(angle_alpha,cv2.ml.ROW_SAMPLE,label_alpha)

In [ ]:
np.shape(label_alpha)
label

In [ ]:
print(np.shape(angle_alpha))
print(angle[0])

In [ ]:
# 숫자
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
recognizeDelay = 0.5
start_time = time.time()
premotion = 0

with mp_hands.Hands(
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue
                       #cv2.flip(image, 1)
        imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image.flags.writeable = False
        results = hands.process(imgRGB)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                input_data = joint_to_angle(hand_landmarks)
                
                data = np.array([input_data],dtype=np.float32)
                        
                res,results,neighbours,dist = knn.findNearest(data,3)
                
                # 손 landmark 그리기
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS)

                index = chr(int(results[0][0])+48)
                if index in actions:
                    
                    #checkmotion = actions_alpha[np.argmax(res)]
                    if len(sentence) > 0:
                        if index != sentence[-1]:
                            if index != premotion:
                                premotion = index
                                start_time = time.time()

                            else:
                                # 시간이 1초가 넘어가면 삽입
                                if time.time() - start_time > recognizeDelay:
                                    sentence.append(index)     
    #                         if actions[np.argmax(res)] != sentence[-1]:
    #                             # 이전 제스쳐와 다르면 시간재기                                 
                    else:
                        sentence.append(index)

                # 5개가 넘어가면 삭제
                if len(sentence) > 10:
                    sentence = sentence[-10:]

                # 사각형으로 그리기
                cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
                cv2.putText(image, ' '.join(sentence),(3,30),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        # 종료 조건
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
# 영어
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
recognizeDelay = 0.5
start_time = time.time()
premotion = 0

with mp_hands.Hands(
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue
                       #cv2.flip(image, 1)
        imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image.flags.writeable = False
        results = hands.process(imgRGB)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:        
                input_data = joint_to_angle_alpha(hand_landmarks)
                
                data = np.array([input_data],dtype=np.float32)
                
                print(np.shape(data))
                
                res,results,neighbours,dist = knn_alpha.findNearest(data,3)
                
                # 손 landmark 그리기
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS)

                index = chr(int(results[0][0])+65)
                print(index)
                print(index in actions_alpha)
                if index in actions_alpha:
                    
                    #checkmotion = actions_alpha[np.argmax(res)]
                    if len(sentence) > 0:
                        if index != sentence[-1]:
                            if index != premotion:
                                premotion = index
                                start_time = time.time()

                            else:
                                # 시간이 1초가 넘어가면 삽입
                                if time.time() - start_time > recognizeDelay:
                                    sentence.append(index)     
    #                         if actions[np.argmax(res)] != sentence[-1]:
    #                             # 이전 제스쳐와 다르면 시간재기                                 
                    else:
                        sentence.append(index)

                # 5개가 넘어가면 삭제
                if len(sentence) > 10:
                    sentence = sentence[-10:]

                # 사각형으로 그리기
                cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
                cv2.putText(image, ' '.join(sentence),(3,30),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        # 종료 조건
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
# 예전거
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
recognizeDelay = 0.5
start_time = time.time()
premotion = 0

with mp_hands.Hands(
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue
                       #cv2.flip(image, 1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image.flags.writeable = False
        results = hands.process(image)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:        
                input_data = np.expand_dims(joint_to_angle(hand_landmarks),axis=0)

                res = model.predict(input_data)
                #print(actions[np.argmax(res)])
                
                # 손 landmark 그리기
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

                if np.argmax(res) > threshold:
                    checkmotion = actions_alpha[np.argmax(res)]
                    
                    if len(sentence) > 0:
                        if checkmotion != sentence[-1]:
                            if checkmotion != premotion:
                                premotion = checkmotion
                                start_time = time.time()

                            else:
                                # 시간이 1초가 넘어가면 삽입
                                if time.time() - start_time > recognizeDelay:
                                    sentence.append(actions_alpha[np.argmax(res)])     
    #                         if actions[np.argmax(res)] != sentence[-1]:
    #                             # 이전 제스쳐와 다르면 시간재기                                 
                    else:
                        sentence.append(actions_alpha[np.argmax(res)])

                # 5개가 넘어가면 삭제
                if len(sentence) > 10:
                    sentence = sentence[-10:]

                # 사각형으로 그리기
                cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
                cv2.putText(image, ' '.join(sentence),(3,30),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        # 종료 조건
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()